In [1]:
from pathlib import Path
import os
import shutil
from datetime import date, timedelta
import xarray as xr
import netCDF4 as nc
import numpy as np
import glob
import dask
import time


In [2]:
train_hrrr_f01 = xr.open_dataarray(f"/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/train_hrrr_alltimes_t2m_f01.nc", decode_timedelta=True)
# train_hrrr_f01_6371200 = xr.open_dataarray(f"/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/train_hrrr_alltimes_t2m_f01_6371200.nc", decode_timedelta=True)

test_hrrr_f01 = xr.open_dataarray(f"/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/test_hrrr_alltimes_t2m_f01.nc", decode_timedelta=True)
# test_hrrr_f01_6371200 = xr.open_dataarray(f"/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/test_hrrr_alltimes_t2m_f01_6371200.nc", decode_timedelta=True)


In [3]:
# train_hrrr_fixed = train_hrrr_f01[:-1]
# train_hrrr_fixed.to_netcdf(f"/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/train_hrrr_alltimes_t2m_f01_fixedindices.nc")

In [4]:
train_hrrr_f01.valid_time.data

array(['2021-01-01T00:00:00.000000000', '2021-01-01T01:00:00.000000000',
       '2021-01-01T02:00:00.000000000', ...,
       '2023-12-31T21:00:00.000000000', '2023-12-31T22:00:00.000000000',
       '2023-12-31T23:00:00.000000000'], dtype='datetime64[ns]')

In [8]:
test_hrrr_f01.valid_time.data

array(['2024-01-01T00:00:00.000000000', '2024-01-01T01:00:00.000000000',
       '2024-01-01T02:00:00.000000000', ...,
       '2024-12-31T21:00:00.000000000', '2024-12-31T22:00:00.000000000',
       '2024-12-31T23:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
# test_hrrr_f01_fixed = xr.concat([train_hrrr_f01[-2:],test_hrrr_f01], dim='sample_idx') #because just [-1] results in scalar sample_idx when array is needed
# test_hrrr_f01_fixed = test_hrrr_f01_fixed[1:]
# test_hrrr_f01_fixed.assign_coords(sample_idx=('sample_idx', [i for i in range(len(test_hrrr_f01_fixed.valid_time.data))]))

In [7]:
# test_hrrr_f01_fixed.to_netcdf(f"/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/test_hrrr_alltimes_t2m_f01_fixedindices.nc")